In [ ]:
%pip install numpy pandas scipy plotly scikit-learn lempel_ziv_complexity ordpy antropy jupytext

In [ ]:
import datetime
from pathlib import Path
import pandas as pd
import numpy as np
import json
from scipy import signal
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ordpy
import antropy

from utils import *
from mt_spectrogram import multitaper_spectrogram, nanpow2db

pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_colwidth', 1000)

DATADIR = Path("data/bob")
for fn in sorted(list(DATADIR.glob("*.json"))): 
    print(fn.name)

In [ ]:
DATAMAP = {
    "eo": "MuseS-5743_2025-05-08T01:47:05.579Z.json",
    "ec": "MuseS-5743_2025-05-08T01:48:20.341Z.json",
    "am": "MuseS-5743_2025-05-08T01:53:04.158Z.json",
}

In [ ]:
dfs = {}
imu_dfs = {}
ppg_dfs = {}
for cond in DATAMAP:
    metadata, eeg_df, motion_df, ppg_df = load_eeg(DATADIR / DATAMAP[cond])
    dfs[cond] = eeg_df
    imu_dfs[cond] = motion_df
    ppg_dfs[cond] = ppg_df

In [ ]:
px.line(dfs["eo"].AF8)

In [ ]:
sdfs = []
for cond, df in dfs.items():
    for e in df.columns:
        f, spec = compute_average_power_spectrum(df[e].values, drop_first_n=1, drop_last_m=1)
        tmpdf = pd.DataFrame({"freq": f, "spec": spec})
        tmpdf["cond"] = cond
        tmpdf["electrode"] = e
        sdfs.append(tmpdf)

specdf = pd.concat(sdfs)
specdf.head()

In [ ]:
px.line(specdf.loc[specdf.electrode=="AF8"], x="freq", y="spec", color="cond")

In [ ]:
bands = {#'Delta': (0, 4),
         'Theta': (4, 8),
         'Alpha': (8, 12),
         'Beta': (12, 30),
         'Gamma': (30, 55),
         'High-gamma': (65, 100)}

eeg_pow = calc_bands_power(dfs["ec"]["AF7"], EEG_DT, bands)
fig = go.Figure(go.Bar(x=[v for v in eeg_pow.values()], y=[k for k in eeg_pow], orientation='h'))
fig.show()

## IMU (motion) analysis

In [ ]:
px.line(imu_dfs["eo"], y=["gyr_x", "gyr_y", "gyr_z"])

## PPG (pulseox) analysis

In [ ]:
ppg_dfs["eo"].head()

In [ ]:
ppg_df = ppg_dfs["eo"]
cols = ["ppg0", "ppg1"]
HEIGHT = 300
peaks = {}
for c in cols:
    ppg_df[f"{c}_filt"] = butter_bandpass_filter(ppg_df[c], .5, 10, PPG_FS, order=6)
    peaks[c], _ = signal.find_peaks(ppg_df[f"{c}_filt"], height=HEIGHT)
    #ppg_df[f"{c}_peaks"] = 0
    #ppg_df.loc[f"{c}_peaks" = 1

In [ ]:
FONTCOLOR = 'rgba(0.4,0.4,0.4,1.0)'
GRIDCOLOR = 'rgba(1.0,1.0,1.0,0.3)'
FONTSIZE = 16
bpm = 60 / np.diff(ppg_df.index[peaks["ppg0"]].values).mean()
fig1 = px.line(ppg_df, y=["ppg0_filt"], color_discrete_sequence=["rgba(.3,.3,.3,.5)", "rgba(.5,.5,.3)"])
fig2 = px.scatter(ppg_df.ppg0_filt.iloc[peaks["ppg0"]], 
                  color_discrete_sequence=["rgba(.6,.4,.1,.7)", "rgba(.5,.5,.3)"])
fig = go.Figure(data=fig1.data + fig2.data)
fig.update_layout(showlegend=False, # xaxis=dict(range=(10, 40)
                  font=dict(size=FONTSIZE, color=FONTCOLOR), 
                  paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)',
                  margin=dict(l=0, r=40, t=10, b=0),
                  xaxis_title="Time (seconds)", yaxis_title="PPG (arb.)",
                  height=300, width=900)
fig.update_yaxes(linecolor='lightgray', zerolinewidth=1, zerolinecolor=GRIDCOLOR, 
                 mirror=False, gridcolor=GRIDCOLOR)
fig.update_xaxes(linecolor='lightgray', zerolinewidth=1, zerolinecolor=GRIDCOLOR, 
                 mirror=False, gridcolor=GRIDCOLOR)
fig.add_annotation(x=11, y=900, text=f"heartrate: {bpm:0.1f} bpm", xanchor="left", yanchor="top", 
                   showarrow=False)
fig

In [ ]:
eeg_df = dfs["eo"]
fig = go.Figure()
Sxx, t, f, meta = multitaper_spectrogram(eeg_df.TP10.values, EEG_FS, freq_range=(0, 80), ncores=-1)

fig.add_trace(go.Heatmap(x=t, y=f, z=Sxx.clip(0, 5), colorscale='Solar'))
fig.update_layout(title='Average Multitaper Spectrogram', 
                  font=dict(size=18),
                  yaxis=dict(title='Frequency (Hz)'), 
                  xaxis=dict(title='Time from start (seconds)'),
                  width=900, height=500)
fig

## EEG Complexity analysis

In [ ]:
ca, cp = lzc(raw.AF7)
print(ca, cp)

In [ ]:
time_series = [logistic(a) for a in [3.05, 3.55, 4]]
time_series += [np.random.normal(size=100000)]

HC = [ordpy.complexity_entropy(series, dx=4) for series in time_series]
HC

In [ ]:
#ordpy.permutation_entropy?
#ordpy.complexity_entropy?

In [ ]:
n = 1000
x = np.sin(np.linspace(0, 100 * np.pi, n)) + np.random.randn(n) * 0.0
c = antropy.lziv_complexity(x)
ce = ordpy.complexity_entropy(x)
print(c, ce)

In [ ]:
win = int(round(8 * EEG_FS))
stepwin = int(round(1 * EEG_FS))
y = raw[EEG_ELECTRODES].rolling(window=win, center=True, step=stepwin).apply(lzc)
y["reltime"] = raw.reltime.groupby(raw.index // stepwin).mean()
px.line(y, x="reltime", y=[])

# Scraps
The rest is likely broken

## EEG Spectrogram

NOTE: this seems to be broken for modern versions of numpy

In [ ]:
idx = 1
EEG_ELECTRODES = dfs[idx].electrode.unique()
raw = dfs[idx].pivot(index=['reltime'], columns=['electrode'], values=['samp']).reset_index()
raw.columns = [c[1] if c[1] != '' else c[0] for c in raw.columns]
raw.dropna(inplace=True)
raw.head()

In [ ]:
fig = go.Figure()
Sxx, t, f, meta = multitaper_spectrogram(raw.AF7.values, EEG_FS, freq_range=(0, 120), ncores=-1)

fig.add_trace(go.Heatmap(x=t, y=f, z=Sxx.clip(-5, 5), colorscale='Solar'))
fig.update_layout(title='Average Multitaper Spectrogram', 
                  font=dict(size=18),
                  yaxis=dict(title='Frequency (Hz)'), 
                  xaxis=dict(title='Time from start (seconds)'))
fig

In [ ]:
NPERSEG = 64
#IDX = (20.0, 120.0, 145.0, 300.0)
IDX = (1.0, 20.0, 25.0, 50.0)

fig = go.Figure()
idx = (raw.reltime > IDX[0]) & (raw.reltime < IDX[1])
f, Cxy = signal.coherence(raw.AF7[idx] + raw.TP9[idx], raw.AF8[idx] + raw.TP10[idx], 256, nperseg=NPERSEG)
fig.add_trace(go.Scatter(x=f, y=Cxy, mode='lines', name=f'Task'))
idx = (raw.reltime > IDX[2]) & (raw.reltime < IDX[3])
f, Cxy = signal.coherence(raw.AF7[idx] + raw.TP9[idx], raw.AF8[idx] + raw.TP10[idx], 256, nperseg=NPERSEG)
fig.add_trace(go.Scatter(x=f, y=Cxy, mode='lines', name=f'Rest'))
    
fig.update_layout(yaxis= {'type': 'log', 'title': 'Coherence'},
                  xaxis_title='Frequency',
                  legend={'font': {'size': 14}, 
                          #'title': {'font': {'size': 16}, 'text': 'Measure'},
                          'yanchor': 'bottom', 'y': 0.05, 'xanchor': 'center', 'x': 0.5},
                  title='Fronto-temporal Coherence',
                  font={'size': 18})

fig.show()

In [ ]:
NPERSEG = 64
#IDX = (20.0, 120.0, 145.0, 300.0)
IDX = (1.0, 16.0, 22.0, 55.0)

fig = go.Figure()
for k in [('E0', 'E1'), ('E3', 'E2')]:
    idx = (raw.reltime > IDX[0]) & (raw.reltime < IDX[1])
    f, Cxy = signal.coherence(raw[k[0]][idx], raw[k[1]][idx], 256, nperseg=NPERSEG)
    fig.add_trace(go.Scatter(x=f, y=Cxy, mode='lines', name=f'Task {k[0]} v. {k[1]}'))
    idx = (raw.reltime > IDX[2]) & (raw.reltime < IDX[3])
    f, Cxy = signal.coherence(raw[k[0]][idx], raw[k[1]][idx], 256, nperseg=NPERSEG)
    fig.add_trace(go.Scatter(x=f, y=Cxy, mode='lines', name=f'Rest {k[0]} v. {k[1]}'))
    
fig.update_layout(yaxis= {'type': 'log', 'title': 'Coherence'},
                  xaxis_title='Frequency',
                  legend={'font': {'size': 14}, 
                          #'title': {'font': {'size': 16}, 'text': 'Measure'},
                          'yanchor': 'bottom', 'y': 0.05, 'xanchor': 'center', 'x': 0.5},
                  title='Coherence',
                  font={'size': 18})

fig.show()

In [ ]:
from numpy_ext import rolling_apply

def coherence(x, y):
    f, Cxy = signal.coherence(x, y, 256, nperseg=NPERSEG)
    return f, Cxy

df = raw.copy().set_index('samp')

#df[['f', 'Cxy']] = rolling_apply(coherence, , df.AF7.values, df.TP9.values)
#locdf[['dist', 'bearing']] = pd.DataFrame(np.row_stack(np.vectorize(dist_az, otypes=['O'])(
#    locdf['latitude'], locdf['longitude'], locdf['homelat'], locdf['homelon'])), index=locdf.index)
#print(df)

In [ ]:
fig = make_subplots(rows=3, cols=1, subplot_titles=('Sensors', 'Raw EEG', 'Muse Bands'))
#fig = go.Figure(go.Bar(y=statdf.index, x=statdf['User-days'], orientation='h'))

for v in ['x', 'y', 'z']:
    fig.add_trace(go.Scatter(x=acc.samp, y=acc[v], name=f'Accel {v.upper()}'), row=1, col=1)
    fig.add_trace(go.Scatter(x=gyr.samp, y=gyr[v], name=f'Gyro {v.upper()}', yaxis='y2'), row=1, col=1)
fig.update_xaxes(title_text="Time", row=1, col=1)
fig.update_yaxes(title_text="Accelerometer (m/s/s)", row=1, col=1, secondary_y=False)
fig.update_yaxes(title_text="Gyro (rad/s)", row=1, col=1, secondary_y=True, anchor='x',
                 overlaying='y', side='right')

for v in ['TP9', 'AF7', 'AF8', 'TP10']: #, 'Aux']:
    fig.add_trace(go.Scatter(x=raw.samp, y=raw[v], name=f'{v.upper()}', opacity=0.5), row=2, col=1)
fig.update_xaxes(title_text="Time", row=2, col=1)

#for v in ['delta', 'theta', 'alpha', 'beta', 'gamma']:
#    tmp = band.loc[bands.band == v, :].copy().reset_index()
#    tmp['samp'] = tmp.index / SEN_FS
#    fig.add_trace(go.Scatter(x=tmp.samp, y=tmp.AF7 + tmp.AF8 + tmp.TP9 + tmp.TP10, name=f'{v}'), row=3, col=1)
#fig.update_xaxes(title_text="Time", row=3, col=1)

fig.update_layout(height=1000, 
                  title='Muse EEG', 
                  #showlegend=False,
                  font={'size': 18})

fig.show() 

In [ ]:
# WORK IN PROGRESS
# may be able to simplify data loading for eeg and ppg
def parse_jsn(jsn, dt, nsamp, seq_name="index", chan_name="electrode"):
    dfs = []

    seq_start = jsn[0][seq_name]
    for d in jsn:
        tmpdf = pd.DataFrame([{"chan": d[chan_name], "value": s,} 
                              for i, s in enumerate(d["samples"])])
        relseq_time = (d["index"] - seq_start) * dt * nsamp
        tmpdf["reltime"] = [dt * i + relseq_time for i in range(nsamp)]
        dfs.append(tmpdf)
    df = pd.concat(dfs).pivot(index="reltime", chan_name="electrode", values="value")